# Выбор локации для скважины

Компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

В моём распоряжении пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски необходимо техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
#импорт библиотек
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Загрузка и подготовка данных

У меня есть три датасета, каждый из которых описывает отдельный регион. Для начала, загружу предоставленные датасеты и сохраню их как data_0,  data_1 и data_2 соответственно. Выведу размер датасетов и пять верхних строк.

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
print(data_0.shape)
data_0.head()

(100000, 5)


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
print(data_1.shape)
data_1.head()

(100000, 5)


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_2.shape)
data_2.head()

(100000, 5)


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Данные у нас подготовленные заранее, и значения столбцов мне не понятны. Вот какая информация прилагается к датасетам:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Проверю наличие дубликатов.

In [5]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


Теперь посмотрим наличие дубликатов среди именно столбца id, в котором предоставлены уникальные идентификаторы скважин.

In [6]:
print(data_0['id'].duplicated().sum())
print(data_1['id'].duplicated().sum())
print(data_2['id'].duplicated().sum())

10
4
4


А вот здесь у нас обнаружилось суммарно 18 продублированных значений. Поскольку я не знаю наверняка, какие данные о каждой скважине правдивые, просто удалю эти строки: на датасеты из 100 тыс. строк это не окажет большого влияния.

In [7]:
data_0 = data_0.drop_duplicates(subset='id').reset_index(drop=True)
data_1 = data_1.drop_duplicates(subset='id').reset_index(drop=True)
data_2 = data_2.drop_duplicates(subset='id').reset_index(drop=True)
print(data_0['id'].duplicated().sum())
print(data_1['id'].duplicated().sum())
print(data_2['id'].duplicated().sum())

0
0
0


Я удалила дубликаты, теперь проверю пропущенные значения.

In [8]:
print(data_0.isna().sum())
print(data_1.isna().sum())
print(data_2.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Пропусков также нет, данные были предварительно хорошо почищены.

### Промежуточный вывод

Я импортировала три датасета, изучила их и убедилась в отсутствии явных дубликатов и пропущенных значений. Удалила неявные дубликаты.

## Обучение и проверка модели

В этом разделе я буду обучать модель по значениям трёх параметров f0, f1, f2 предсказывать объём сырья product.

### Создание выборок

Чтобы создать обучающую и валидационную выборки, поделю три датасета на обучающую и валидационную выборки в соотношении 3:1.

In [9]:
data_train_0, data_valid_0 = train_test_split(data_0, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки региона 0:', data_train_0.shape)
print('Размер валидационной выборки региона 0:', data_valid_0.shape)

Размер тренировочной выборки региона 0: (74992, 5)
Размер валидационной выборки региона 0: (24998, 5)


In [10]:
data_train_1, data_valid_1 = train_test_split(data_1, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки региона 1:', data_train_1.shape)
print('Размер валидационной выборки региона 1:', data_valid_1.shape)

Размер тренировочной выборки региона 1: (74997, 5)
Размер валидационной выборки региона 1: (24999, 5)


In [11]:
data_train_2, data_valid_2 = train_test_split(data_2, test_size=0.25, random_state=12345)
print('Размер тренировочной выборки региона 2:', data_train_2.shape)
print('Размер валидационной выборки региона 2:', data_valid_2.shape)

Размер тренировочной выборки региона 2: (74997, 5)
Размер валидационной выборки региона 2: (24999, 5)


Я успешно создала выборки data_train и data_valid. Теперь разделю их на features и target.

In [12]:
features_train_0 = data_train_0.drop(['product'], axis=1)
features_train_0 = features_train_0.drop(['id'], axis=1)
target_train_0 = data_train_0['product']
features_valid_0 = data_valid_0.drop(['product'], axis=1)
features_valid_0 = features_valid_0.drop(['id'], axis=1)
target_valid_0 = data_valid_0['product']
print('Размер features тренировочной выборки 0:', features_train_0.shape)
print('Размер target тренировочной выборки 0:', target_train_0.shape)
print('Размер features валидационной выборки 0:', features_valid_0.shape)
print('Размер target валидационной выборки 0:', target_valid_0.shape)

Размер features тренировочной выборки 0: (74992, 3)
Размер target тренировочной выборки 0: (74992,)
Размер features валидационной выборки 0: (24998, 3)
Размер target валидационной выборки 0: (24998,)


In [13]:
features_train_1 = data_train_1.drop(['product'], axis=1)
features_train_1 = features_train_1.drop(['id'], axis=1)
target_train_1 = data_train_1['product']
features_valid_1 = data_valid_1.drop(['product'], axis=1)
features_valid_1 = features_valid_1.drop(['id'], axis=1)
target_valid_1 = data_valid_1['product']
print('Размер features тренировочной выборки 1:', features_train_1.shape)
print('Размер target тренировочной выборки 1:', target_train_1.shape)
print('Размер features валидационной выборки 1:', features_valid_1.shape)
print('Размер target валидационной выборки 1:', target_valid_1.shape)

Размер features тренировочной выборки 1: (74997, 3)
Размер target тренировочной выборки 1: (74997,)
Размер features валидационной выборки 1: (24999, 3)
Размер target валидационной выборки 1: (24999,)


In [14]:
features_train_2 = data_train_2.drop(['product'], axis=1)
features_train_2 = features_train_2.drop(['id'], axis=1)
target_train_2 = data_train_2['product']
features_valid_2 = data_valid_2.drop(['product'], axis=1)
features_valid_2 = features_valid_2.drop(['id'], axis=1)
target_valid_2 = data_valid_2['product']
print('Размер features тренировочной выборки 2:', features_train_2.shape)
print('Размер target тренировочной выборки 2:', target_train_2.shape)
print('Размер features валидационной выборки 2:', features_valid_2.shape)
print('Размер target валидационной выборки 2:', target_valid_2.shape)

Размер features тренировочной выборки 2: (74997, 3)
Размер target тренировочной выборки 2: (74997,)
Размер features валидационной выборки 2: (24999, 3)
Размер target валидационной выборки 2: (24999,)




### Обучение моделей

Теперь обучу три модели линейной регрессии, отдельно для каждого региона, и проверю их на валидационных выборках.

In [15]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)
print('RMSE модели 0 =', mean_squared_error(target_valid_0, predictions_valid_0)**0.5)

RMSE модели 0 = 37.853527328872964


In [16]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)
print('RMSE модели 1 =', mean_squared_error(target_valid_1, predictions_valid_1)**0.5)

RMSE модели 1 = 0.8920592647717029


In [17]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)
print('RMSE модели 2 =', mean_squared_error(target_valid_2, predictions_valid_2)**0.5)

RMSE модели 2 = 40.07585073246016


Сами по себе эти цифры мало о чём говорят, нужно сравнить RMSE с общим разбросом целевого признака в target_valid.

In [18]:
print('Разброс:', target_valid_0.max() - target_valid_0.min())
print('RMSE 0 составляет',
      round((mean_squared_error(target_valid_0, predictions_valid_0)**0.5) * 100 /
            (target_valid_0.max() - target_valid_0.min()), 2),
      'процентов от разброса')

Разброс: 185.3156522711293
RMSE 0 составляет 20.43 процентов от разброса


20% отклонения — это не мало, но не слишком критично. Посмотрим на другие модели.

In [19]:
print('Разброс:', target_valid_1.max() - target_valid_1.min())
print('RMSE 1 составляет',
      round((mean_squared_error(target_valid_1, predictions_valid_1)**0.5) * 100 /
            (target_valid_1.max() - target_valid_1.min()), 2),
      'процентов от разброса')

Разброс: 137.94540774090564
RMSE 1 составляет 0.65 процентов от разброса


У модели для региона 1 по какой-то причине RMSE совсем низкий. Теперь модель 2:

In [20]:
print('Разброс:', target_valid_2.max() - target_valid_2.min())
print('RMSE 2 составляет',
      round((mean_squared_error(target_valid_2, predictions_valid_2)**0.5) * 100 /
            (target_valid_2.max() - target_valid_2.min()), 2),
      'процентов от разброса')

Разброс: 190.02523234331147
RMSE 2 составляет 21.09 процентов от разброса


Тут результат примерно схож с моделью 0. Посмотрим, какой средний запас предсказанного сырья нам дают модели.

In [21]:
print('Средний предсказанный запас для региона 0: ', round(predictions_valid_0.mean(), 2))
print('Средний предсказанный запас для региона 1: ', round(predictions_valid_1.mean(), 2))
print('Средний предсказанный запас для региона 2: ', round(predictions_valid_2.mean(), 2))

Средний предсказанный запас для региона 0:  92.79
Средний предсказанный запас для региона 1:  69.18
Средний предсказанный запас для региона 2:  94.87


По условиям задачи, 1 тысяча баррелей приносит прибыль в 450 тысяч рублей. Т.е. 92.79 тысяч баррелей в перспективе принесут 41 755 500 рублей. 69.18 тысяч баррелей — 31 131 000 рублей, а 94.87 тысяч — 42 691 500 рублей.

### Промежуточный вывод

В данном разделе я разделила все три датасета на обучающую и валидационную выборки. После чего обучила модели линейной регрессии, проверила их на валидационной выборке. Расхождение прогнозируемых значений от фактических я оценила при помощи метрики RMSE, для регионов 0 и 2 оно составило чуть больше 20% от разброса данных, для региона 1 меньше одного процента.

## Подготовка к расчёту прибыли

Теперь нужно определить, сколько минимально сырья должна выдавать скважина, чтобы окупить свою стоимость. Для начала, сохраню переменные: price — цена за 1 тысячу баррелей, total_budget — всего бюджета на разработку в регионе.

In [22]:
PRICE = 450000
TOTAL_BUDGET = 10_000_000_000

В дальнейшем в регионе будет выбрано 200 точек, т.е. для определения бюджета на одну скважину нужно поделить общий бюджет на это колличество. Результат сохраню в budget.

In [23]:
budget = TOTAL_BUDGET / 200
budget

50000000.0

Итого, на одну скважину мы можем выделить 50 миллионов. Подсчитаю, сколько минимально нужно добыть сырья, чтобы окупить расходы на скважину.

In [24]:
min_product = budget / PRICE
min_product

111.11111111111111

Одна скважина должна принести минимум 111,1 тысяч баррелей, чтобы окупить затраты на свою разработку. Сравним эти цифры со средними значениями продуктивности в каждом регионе.

In [25]:
print('Разность средней продуктивности и минимальных затрат в регионе 0:', data_0['product'].mean() - min_product)
print('Разность средней продуктивности и минимальных затрат в регионе 1:', data_1['product'].mean() - min_product)
print('Разность средней продуктивности и минимальных затрат в регионе 2:', data_2['product'].mean() - min_product)

Разность средней продуктивности и минимальных затрат в регионе 0: -18.61142689336758
Разность средней продуктивности и минимальных затрат в регионе 1: -42.287195193070474
Разность средней продуктивности и минимальных затрат в регионе 2: -16.11276899177733


### Промежуточный вывод

Каждая скважина из двух сотен должна принести хотя бы 111,1 тысяч баррелей, чтобы окупить затраты. При этом, в предыдущем разделе я уже посчитала среднюю добычу: 92.79, 69.18, 94.87 тысяч баррелей. Т.е. в каждом регионе, особенно в регионе 1, бОльшая часть скважин не окупают свои вложения.

## Расчёт прибыли и рисков

### Функция подсчёта прибыли

Напишу функцию, которая будет считать прибыль с заданного числа самых продуктивных скважин. Для этого сортирую список скважин по убыванию, выбираю верхние заданного колличества, суммирую между собой и умножаю на 450,000 — стоимость тысячи баррелей. Вычитаю расход на постройку — 50 миллионов рублей.

In [26]:
def profit(target, pred, number=1):
    pred_sorted = pred.sort_values(ascending=False)
    selected = target[pred_sorted.index][:number]
    return selected.sum() * PRICE - TOTAL_BUDGET

### Bootstrap
Далее, применю технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли в этих регионах.

In [27]:
values_0 = []
state = np.random.RandomState(12345) #задаём рандом_стэйт для повторяемости результатов
target_valid_0 = target_valid_0.reset_index(drop=True) #Индексы в таргет_валид могут превышать 25 тыс. Обновляем их, чтобы упорядочить
predictions_valid_0 = pd.Series(predictions_valid_0) #Переводим предсказания в Series из списка, чтобы по индексам сопоставлять в методе выше
for i in range(1000): #bootstrap региона 0
    target_0_sub = target_valid_0.sample(n=500, replace=True, random_state=state)
    pred_0_sub = predictions_valid_0[target_0_sub.index]
    values_0.append(profit(target_0_sub, pred_0_sub, 200))

In [28]:
values_1 = []
target_valid_1 = target_valid_1.reset_index(drop=True)
predictions_valid_1 = pd.Series(predictions_valid_1)
for i in range(1000): #bootstrap региона 1
    target_1_sub = target_valid_1.sample(n=500, replace=True, random_state=state)
    pred_1_sub = predictions_valid_1[target_1_sub.index]
    values_1.append(profit(target_1_sub, pred_1_sub, 200))

In [29]:
values_2 = []
target_valid_2 = target_valid_2.reset_index(drop=True)
predictions_valid_2 = pd.Series(predictions_valid_2)
for i in range(1000): #bootstrap региона 2
    target_2_sub = target_valid_2.sample(n=500, replace=True, random_state=state)
    pred_2_sub = predictions_valid_2[target_2_sub.index]
    values_2.append(profit(target_2_sub, pred_2_sub, 200))

### Средняя прибыль и доверительный интервал
Далее мне надо найти среднюю прибыль скважины в регионе, 95%-й доверительный интервал и риск убытков. Все эти показатели я буду сравнивать. Начну со средней прибыли и доверительного интервала.

In [30]:
values_0 = pd.Series(values_0) #перевод в Series для операций, недоступных для list
values_1 = pd.Series(values_1)
values_2 = pd.Series(values_2)

profit_0_mean = values_0.mean()
print('Среднее региона 0 равно', profit_0_mean)

interval_0 = [values_0.quantile(0.025), values_0.quantile(0.975)]

print("95%-ый доверительный интервал равен", interval_0)

Среднее региона 0 равно 409428038.62143606
95%-ый доверительный интервал равен [-131536028.70166382, 944395582.7546725]


In [31]:
profit_1_mean = values_1.mean()
print('Среднее региона 1 равно', profit_1_mean)

interval_1 = [values_1.quantile(0.025), values_1.quantile(0.975)]

print("95%-ый доверительный интервал равен", interval_1)

Среднее региона 1 равно 536400199.43510306
95%-ый доверительный интервал равен [112954247.12370124, 998504156.6468805]


In [32]:
profit_2_mean = values_2.mean()
print('Среднее региона 2 равно', profit_2_mean)

interval_2 = [values_2.quantile(0.025), values_2.quantile(0.975)]

print("95%-ый доверительный интервал равен", interval_2)

Среднее региона 2 равно 339478034.1977997
95%-ый доверительный интервал равен [-224089221.7440758, 847067587.6863929]


Самые высокие среднее и доверительный интервал у региона 1. Отстаёт больше всего 2 регион.

In [33]:
pred_loss_0 = values_0.loc[values_0 < 0]
print('Риск убытков среди предсказаний в регионе 0:',
      round(len(pred_loss_0) * 100 / len(values_0), 3), '%')

Риск убытков среди предсказаний в регионе 0: 7.1 %


In [34]:
pred_loss_1 = values_1.loc[values_1 < 0]
print('Риск убытков среди предсказаний в регионе 1:',
      round(len(pred_loss_1) * 100 / len(values_1), 3), '%')

Риск убытков среди предсказаний в регионе 1: 0.3 %


In [35]:
pred_loss_2 = values_2.loc[values_2 < 0]
print('Риск убытков среди предсказаний в регионе 2:',
      round(len(pred_loss_2) * 100 / len(values_2), 3), '%')

Риск убытков среди предсказаний в регионе 2: 11.8 %


По условию задачи, необходим риск убытков ниже 2,5%, этому соответствует только регион 1 с риском 0,3%.

### Промежуточный вывод

Самый прибыльный регион под номером 1, у него самая высокая средняя прибыль, доверительный интервал, и минимальные риски убытка.

## Вывод


В данном проекте по машинному обучению задачей было определить регион с наибольшей ожидаемой прибылью.

Для выполнения данной задачи я произвела следующие действия:

- Загрузила и изучила данные;
- Удалила неявные дубликаты;
- Обучила три модели линейной регрессии определять продуктивность в каждом регионе;
- Сохранила константы по условию задачи;
- Написала функцию подсчёта прибыли по внесённым данным о продуктивности и колличеству прибыльных скважин;
- Применила метод Bootstrap для определения распределения ожидаемой прибыли в каждом из трёх регионов;
- Определила среднее и 95% доверительный диапазон;
- Посчитала риски убытков;
- Выделила самый прибыльный регион.

Я обучила модель линейной регрессии, которая предсказала продуктивность скважин в каждом из трёх регионов, и выбрала регион номер 1 (по рассчёту от 0 до 2), в котором ожидается наибольшая продуктивность.